# Importing experimental data

Raw data can be of diverse lab-specific formats

In [ ]:
%load_ext param.ipython
import panel as pn

Initialize the larvaworld registry. This loads some components from disc and builds the rest on the fly.

We also set VERBOSE=1 to get more info

In [ ]:
import larvaworld
from larvaworld.lib import reg

larvaworld.VERBOSE = 1

We will start with the *LabFormat* class which supports importing lab-specific data formats

In [ ]:
from larvaworld.lib.reg.generators import LabFormat

%params LabFormat

Let's generate a new instance

In [ ]:
lf_new = LabFormat(labID="MyLab")
print(f"An instance of {lf_new.__class__}")


%params lf_new

Stored instances of the *LabFormat* class are available through the configuration registry.

The registry is retrieved from a dictionary of registry objects by the *LabFormat* key.

In [5]:
LFreg = reg.conf.LabFormat

Each lab-specific data-format configuration is stored in the registry's dictionary under a unique ID.

Let's print the IDs

In [ ]:
lfIDs = LFreg.confIDs
print(f"The IDs of the stored configurations of LabFormat class are :{lfIDs}")

# The registry is supported by a nested dictionary :
LFdict = LFreg.dict

# The path where the dictionary is stored:
print(LFreg.path_to_dict)


# The configuration IDs are the keys. They correspond to a nested dictionary :
lfID = lfIDs[0]
lf0_entry = LFdict[lfID]
print()
print(f"An instance of {lf0_entry.__class__}")

# The configuration dictionary can be retrieved directly by :
lf0_entry2 = LFreg.getID(lfID)
print()
print(lf0_entry == lf0_entry2)

In [ ]:
# The configuration object can be retrieved directly by :
lf0 = LFreg.get(lfID)
print(f"The object under the ID : {lfID} is an instance of {lf0.__class__}")
print()

%params lf0

In [ ]:
pn.Param(lf0)

In [ ]:
# The configuration dictionary can be retrieved directly from the object :
lf0_entry3 = lf0.nestedConf

# As well as the parameter keys
print(lf0.param_keys)
print()

# The path where the lab data are stored:
print(lf0.path)
# print(lf0.raw_folder)

import

In [ ]:
id = "Schleyer"
Schleyer_lf = LFreg.get(id)
# print(Schleyer_lf.labID)
# print(LFentry.tracker)

%params Schleyer_lf.tracker

In [11]:
kws0 = {
    #'labID': 'Schleyer',
    "group_id": "exploration",
}

# Merged case
N = 40
kws1 = {
    "parent_dir": "exploration",
    "merged": True,
    "N": N,
    "min_duration_in_sec": 120,
    "refID": f"exploration.{N}controls",
    **kws0,
}

# Single dish case
folder = "dish01"
kws2 = {
    "parent_dir": f"exploration/{folder}",
    # 'merged': False,
    # 'N': None,
    "min_duration_in_sec": 90,
    "id": folder,
    "refID": f"exploration.{folder}",
    **kws0,
}

In [ ]:
d1 = Schleyer_lf.import_dataset(**kws1)
d2 = Schleyer_lf.import_dataset(**kws2)

print()
print(f"The object having the ID : {d1.id} is an instance of {d1.__class__}")

In [ ]:
s, e, c = d1.data

# Timeseries df
s.head()

In [ ]:
# Endpoint df
e

In [ ]:
# Dataset metadata/configuration
print(f"The dataset configuration is an instance of {c.__class__}")

print(c.param_keys)

In [ ]:
%params reg.gen.LabFormat

In [ ]:
# s = s.astype(float)
s.dtypes